In [57]:
class Applicant:
    def __init__(self, name, preferences):
        self.name = name
        self.preferences = preferences
        self.proposal_index = 0  # Индекс в списке предпочтений, куда была сделана предложение
        self.partner = None  # Проект, с которой студент образует пару

    def make_proposal(self):
        # Метод для сделать предложение выбранным проектом из списка предпочтений
        preference = self.preferences[self.proposal_index]
        self.proposal_index += 1
        return preference

    def receive_partner(self, partner):
        # Метод для получения проекта
        self.partner = partner

    def is_free(self):
        # Метод, возвращающий True, если студент свободен и еще не образовал пару
        return self.partner is None


class Project:
    def __init__(self, name, capacity, preferences):
        self.name = name
        self.capacity = capacity  # количество
        self.preferences = preferences  # предпочтения проекта
        self.applicants = []

    def add_applicant(self, applicant):
        # Метод для добавления студента в список принятых проектом
        self.applicants.append(applicant)

    def remove_applicant(self, applicant):
        # Метод для удаления студента из списка принятых проектом
        self.applicants.remove(applicant)

    def has_capacity(self):
        # Метод, возвращающий True, если в проекте есть свободные места
        return len(self.applicants) < self.capacity

    def prefers_applicant(self, applicant):
        # Метод, проверяющий, предпочитает ли проекта данный студент
        return applicant in self.preferences[:self.capacity]

    def get_least_preferred_applicant(self):
        # Метод, возвращающий наименее предпочитаемого студента в проекте
        return self.applicants[-1]


def gale_shapley(applicants, schools):
    while True:
        free_applicants = [applicant for applicant in applicants if applicant.is_free()]

        if not free_applicants:
            break

        for applicant in free_applicants:
            school_name = applicant.make_proposal()
            school = schools[school_name]

            if school.has_capacity():
                school.add_applicant(applicant)
                applicant.receive_partner(school)
            else:
                least_preferred_applicant = school.get_least_preferred_applicant()

                if school.prefers_applicant(applicant):
                    school.remove_applicant(least_preferred_applicant)
                    least_preferred_applicant.receive_partner(None)

                    school.add_applicant(applicant)
                    applicant.receive_partner(school)

    matches = [(applicant.name, applicant.partner.name) for applicant in applicants]
    return matches

In [58]:
# Создание студентов

applicant1 = Applicant("Студент1", ["Проект1", "Проект2", "Проект3"])
applicant2 = Applicant("Студент2", ["Проект2", "Проект1", "Проект3"])
applicant3 = Applicant("Студент3", ["Проект3", "Проект2", "Проект1"])
applicant4 = Applicant("Студент4", ["Проект3", "Проект2", "Проект1"])
applicant5 = Applicant("Студент5", ["Проект2", "Проект3", "Проект1"])

# Создание проекта
school1 = Project("Проект1", 2, ["Студент1", "Студент2", "Студент3", "Студент4", "Студент5"])
school2 = Project("Проект2", 1, ["Студент2", "Студент1", "Студент4", "Студент3", "Студент5"])
school3 = Project("Проект3", 2, ["Студент2", "Студент4", "Студент3", "Студент1", "Студент5"])

# Списки студентов и проектов
applicants = [applicant1, applicant2, applicant3, applicant4, applicant5]
schools = {"Проект1": school1, "Проект2": school2, "Проект3": school3}

# Выполнение алгоритма Гейла-Шепли
matches = gale_shapley(applicants, schools)

matches

[('Студент1', 'Проект1'),
 ('Студент2', 'Проект2'),
 ('Студент3', 'Проект3'),
 ('Студент4', 'Проект3'),
 ('Студент5', 'Проект1')]